In [2]:
from functools import reduce
import md5_params

In [3]:
def H_md5(a,b,c,d,message_part):
    #print(1)
    for i in range(4):
        for j in range(16):
            g_b_c_d = md5_params.g_functions[i](b,c,d)
            X_k = int.from_bytes(message_part[4*md5_params.indexs_function_per_interation[i](j):4*md5_params.indexs_function_per_interation[i](j)+4],byteorder="little")
            temp1 = a + g_b_c_d + X_k + md5_params.T[i*16+j]
            temp2 = b + md5_params.left_rotate(temp1,md5_params.s[i*16+j])
            a , b , c , d = d , temp2 & 0xFFFFFFFF , b ,c
    return a , b ,c ,d

In [4]:
def md5(message):
    
    message = bytearray(message,encoding="utf-8")
    if len(message) < 8:
        lack_length = 8-len(message)
        remaind_padding = bytearray("0",encoding="utf-8")
        for i in range(8-1-len(message)):
            remaind_padding.append(0x00)
        remaind_padding+=message[-lack_length:]
    else:
        remaind_padding=message[-8:]
    #print(remaind_padding)
    if 8*len(message) == 448:
        message.append(0x80)
        for i in range(63):
            message.append(0x00)
    else:
        message.append(0x80)
        while len(message) % 64 !=56:
            #print(len(message))
            message.append(0x00)
    message+=remaind_padding
    a,b,c,d = md5_params.IV
    #print(len(message))
    for x in range(len(message)//64):
        a,b,c,d = H_md5(a,b,c,d,message[x*64:x*64+64])
    #print(len(a))
    return reduce(lambda x,y:x+y,map(lambda x : (x&0xFFFFFFFF).to_bytes(4,byteorder="little"),[a,b,c,d]))

In [5]:
if __name__ == "__main__":
    text = input("inout the text you want digest")
    print("the byte format output is {}".format(md5(text)))
    print("the bytearray format output is {}".format('{:032x}'.format(int.from_bytes(md5(text),byteorder="little"))))

inout the text you want digest								Web  安全 assignment 2 1.	算法原理概述 首先这次报告要是是实现的是md5这一个消息摘要的算法，所谓消息摘要就是说相当于是对于整个一个信息做一个hash，但是这一个hash应该要能够尽可能包括这一个信息的所有信息，换句话来说就是应该能够做到哪怕修改了一个byte，最后得到的摘要的结果也都会非常不同。 这样做的好处也很明显，通过这个信息摘要就能够验证文件的完整性和利用摘要的单向性来起到验证的作用，比较常用在密码的存储上，通常还有加盐之类的混淆操作。 至于算法是怎么实现的，说白了其实就是把信息切分成512bit的组，再轮流对不同的组和上一次迭代得到的中间变量进行hash，最后的结果就是能够得到包括了整个信息的摘要。   整个流程就在这里了。一共就两部分 a)	第一部分就是padding没什么好说的 b)	第二部分就是hash，是用128bit的中间变量和512bit的信息块不断做hash，最后得到的结果是128bit的摘要，其中hash过程分为4轮一共64次迭代，每一轮所用到的一些函数和移位的位置都不同，也都是为了起到混淆的作用。 2.	代码分析 首先要说明的是这一次实现采用的语言是python，没什么好说的主要就是快，用起来爽。 1）	整个的代码就是分成了两块，一个文件专门用来存储预先定义的变量和不同的hash中间函数，因为是和整个md5主题无关，而且独立出来还能够有助于对于这一些预先定义的变量进行修改来达到定制的效果。 2）	另一个主体文件里面就是核心的md5结构，其中把hash_md5，这一个专门用来hash中间信息块的函数专门提取出来成独立的函数。   这就是预先定义的变量的文件，没什么好说的，就是完全按照老师ppt上的参数来设置的。   这就是md5对于每一个512bit的消息块所进行hash的函数，其中调用的都是params文件下的参数，也都没有什么好说的，唯一值得注意的就是因为设计到byte的加法，所以需要&0xFFFFFFFF这一个取余操作，防止越界，因为是要模拟寄存器的操作，所以对于bit1的位数有明确的限制。   这就是整个md5的算法流程，输入就是普通的字符串但是通过转换成bytearray，来能够在byte级别的操作。 其中遇到的一个问题但是ppt上没